In [1]:
#encoding='utf-8'
#百度贴吧爬虫 获取每个帖子的用户ID,评论内容，时间
#路径 /DB 分页内容     /FILES URL数据表
#cfy2400@foxmail.com 方
#PS:仅适用新版贴吧

In [2]:
import os
import re
import sys
import xlwt
import xlrd
import requests
import time
import math
import queue
import threading
import json
from bs4 import BeautifulSoup

In [3]:
class spyThread(threading.Thread):
    '''
        -level 内容详细程度
        -queue 队列
        -number 爬虫线程数
        -check 爬取链接或页面内容
        -url 要爬取的链接地址
        -name 数据保存命名
    '''
    def __init__(self,level,queue,queue1,number,check,url,name):
        threading.Thread.__init__(self)
        self.l=level
        self.q=queue
        self.q1=queue1
        self.c=check
        self.u=url
        self.name=name
        try:
            self.n=number
        except:
            self.n=-1

    def run(self):
        count=0
        if self.c=='save_content':
            #从excel表中依次读取URL分别访问，将每个页面的内容分别保存至excel
            while not self.q.empty():
                url=self.q.get()
                url_1=url+'?pn='#拼接参数pn
                book=xlwt.Workbook()#新建一个excel文件
                sheet=book.add_sheet('sheet1')#新建一个工作簿
                urlf=url_1+'1'#第一次建立链接的地址
                res=requests.get(urlf)#建立第一次URL连接
                soup=BeautifulSoup(res.text,'html.parser')#将网页存入soup
                #try :
                try:
                    tit=soup.findAll('h3',('core_title_txt'))[0].text#抓取标题
                except:
                    tit=soup.findAll('h1',('core_title_txt'))[0].text#抓取标题
                max_page=soup.findAll('span',('red'))[1].text#获取最大页数
                for j in range(1,int(max_page)+1):
                    url=url_1+str(j)#循环赋值URL抓取多页数据
                    res1=requests.get(url)#建立URL连接
                    soup1=BeautifulSoup(res1.text,'html.parser')#将网页存入soup
                    data=get_data(soup1)#爬取数据
                    save_xl(data,j,sheet,tit,urlf)#写入excel
                if os.path.exists('./db/'+self.name)==0:
                    os.mkdir('./db/'+self.name)

                road=os.getcwd()+'\\db\\'+self.name+'\\'+validateTitle(tit)+'.xls'
                book.save(road)#保存文件
                #print(road+'【保存完毕】')
                #except:
                    #print ('页面',url,'抓取错误')
        elif self.c=='save_url':
            #将主链接下的所有子链接存入队列
            while not self.q.empty():
                url=self.u
                pn=self.q.get()
                url1=url+str(pn*50)
                #print(url1)
                try:
                    soup=get_html(url1)
                    rs=soup.findAll('a',{'class':'j_th_tit'})
                    for k in rs:
                        
                        href='http://tieba.baidu.com/'+k.get('href')
                        tit=k.text
                        self.q1.put([tit,href])
                    #print('第%d页爬取完成'%pn)
                except:
                    print (url1,'数据写入错误')
        else:
            return 0
        
    def get_result(self):
        return self.q1
def validateTitle(title):
    #转换标题中的非法字符
    rstr = r"[\/\\\:\*\?\"\<\>\|]"  # '/ \ : * ? " < > |'
    new_title = re.sub(rstr, "_", title)  # 替换为下划线
    return new_title

def get_data(soup):
    #获取数据保存在字典中
    name=[]
    tim=[]
    content=[]
    item=[]
    
    
    '''
    count=len(soup.findAll('div',{'j_l_post'}))
    for i in range(0,count):#提取时间

        t=json.loads(soup.findAll('div',{'j_l_post'})[i].get('data-field'))['content']['date']#时间
        print(t)
        tim.append(t)
    
    '''
    
    try:
        count=len(soup.findAll('div',{'j_l_post'}))
        for i in range(0,count):#提取时间
            t=json.loads(soup.findAll('div',{'j_l_post'})[i].get('data-field'))['content']['date']#时间
            tim.append(t)
            #print('A:',t)
    except:
        times=soup.findAll('span',('tail-info'))#时间
        for i in range(0,len(times)):#提取时间
            t=(times[i].text)
            try:#判断多个相同标签下哪个字段是时间
                if ":" in t:
                    time.strptime(t, "%Y-%m-%d %H:%M")
                    tim.append(t)
                #print('B:',t)
            except:
                return 0

    names=soup.findAll('a',('p_author_name'))#用户名称
    contents=soup.findAll('div',('d_post_content'))#内容
    for i in range(0,len(names)):#提取用户名称
        n=(names[i].text)
        name.append(n)
    for i in range(0,len(contents)):#提取评论内容
        c=(contents[i].text.strip())
        c=c.rstrip()
        content.append(c)
    #print(len(name),len(tim),len(content))
    for i in range(0,len(name)):#将列表转换为字典
        dic={}
        dic.update(name=name[i],time=tim[i],content=content[i])
        item.append(dic)
    return item

def spy_xlurl_to_xl(name,maxpage):
    #将现有excel中的链接批量访问抓取数据，并保存至excel中
    url_q=queue.Queue()           #网址队列
    threadCount=10                #线程数量
    level=1                       #内容级别
    maxpn=maxpage*50+1            #最大URL数量
    check='save_content'
    try:
        workbook=xlrd.open_workbook('./files/'+name+'.xls')
    except:
        print('未找到表')
        return False
    f=workbook.sheet_by_index(0)
    trows=f.nrows
    time1=time.time()
    if trows:
        if maxpn>trows:
            maxpn=trows
        #print(trows,maxpn)
        for i in range(0,maxpn):     #控制最大URL数量
            url_q.put(f.cell(i,1).value)   #将URL放进队列
        for i in range(threadCount):  
            t=spyThread(level,url_q,'',i,check,'',name)
            t.setDaemon(True)
            t.start()
            #print ('线程%d开始'%i)
        print('爬取数据中...')
        for i in range(threadCount):
            t.join()
            #print ('线程%d结束'%i)
        print('爬虫工作结束，共用时:',time.time()-time1,'s')
        print ('Done')
    else:
        print('未获取到有效URL')
    
def save_xl(data,page,sheet,tit,url):
    #接收数据.当前页数.工作表
    minlines=(page-1)*30+1
    maxlines=(page-1)*30+len(data)+1
    j=0   #控制data数组下标
    if page==1:
        #当页面为第一页时填写表头
        sheet.write(0,0,'序号')
        sheet.write(0,1,'用户ID')
        sheet.write(0,2,'时间')
        sheet.write(0,3,'内容')
        sheet.write(0,4,url)
    for i in range(minlines,maxlines):
        #依次写入数据到excel
        sheet.write(i,0,i)
        sheet.write(i,1,data[j]['name'])
        sheet.write(i,2,data[j]['time'])
        sheet.write(i,3,data[j]['content'])
        j+=1
        
def save_url(q,name):
    #将爬取到的URL存入excel
    book=xlwt.Workbook()
    sheet=book.add_sheet('sheet1',cell_overwrite_ok=True)
    j=0
    while not q.empty():
        ar=q.get()
        tit=ar[0]
        href=ar[1]
        sheet.write(j,0,tit)
        sheet.write(j,1,href)
        j+=1
        #print(tit+' 【完毕】')
    book.save('./files/'+name+'.xls')
    return j
        
def get_html(url):
    #请求URL,返回HTML页面
    res=requests.get(url)
    soup=BeautifulSoup(res.text,"html.parser")
    res.close()
    return soup

def spy_url_save(url,name,maxpage):
    #爬取所有子链接到表
    q=queue.Queue()               #页面数队列
    q1=queue.Queue()              #临时存放URL
    threadCount=10                #线程数量
    level=1                       #内容级别
    check='save_url'
    time1=time.time()
    for i in range(0,maxpage):     #控制最大页面数量
        q.put(i)
    for i in range(threadCount):  
        t=spyThread(level,q,q1,i,check,url,name)
        t.setDaemon(True)
        t.start()
        #print ('线程%d开始'%i)
    print('爬取数据中...')
    for i in range(threadCount):
        t.join()
        #print ('线程%d结束'%i)
    print('爬取完毕，写入excel数据中...')
    count=save_url(q1,name)
    print ('已将所有URL写入excel')
    print('爬虫工作结束，共计',count,'条数据，共用时:',time.time()-time1,'s')

In [7]:
if __name__=='__main__':
    #start=1 爬取链接 start=2 分页爬取页面内容
    start=2

    name='复仇者联盟'
    url='https://tieba.baidu.com/f?kw='+name+'&ie=utf-8&pn='
    if start==1:
        #需要爬取的页面数，每页50个子链接
        maxpage=10
        spy_url_save(url,name,maxpage)#爬取主页面下的子链接存入excel
    elif start==2:
        #需要爬取的页面数，每页50个子链接
        maxpage=1
        spy_xlurl_to_xl(name,maxpage)#将excel中的链接读取分别访问爬取内容，并单独保存为excel
    else:
        print('hello')

爬取数据中...
爬虫工作结束，共用时: 24.249629259109497 s
Done
